(**Click the icon below to open this notebook in Colab**)

[![Open InColab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/xiangshiyin/machine-learning-for-actuarial-science/blob/main/2025-spring/week04/notebook/demo.ipynb)

# Softmax regression vs. Logistic Regression

Last week, we talked about an example of using softmax regression to recognize handwritten digits. The softmax regression algorithm could essentially be expressed as

$$
\begin{aligned}
\hat{y} = \text{softmax}(\mathbf{X} \cdot \theta) = \frac{\exp(\mathbf{X} \cdot \theta)}{\sum_{i=1}^{C} \exp(\mathbf{X}_i \cdot \theta)}
\end{aligned}
$$

Where:
- $X$ is the input training data matrix (shape $(N,D)$), where $N$ is the number of samples in the training batch and $D$ is the number of features.
- $\theta$ is the weight matrix (shape $(D,C)$), where $C$ is the number of classes.
- $\hat{y}$ is the predicted probability distribution over the classes (shape $(N,C)$).


In case $C=2$, we can simplify the softmax regression to logistic regression, which is a special case of softmax regression. The logistic regression algorithm could be expressed as
$$
\begin{aligned}
\hat{y} = \text{sigmoid}(\mathbf{X} \cdot \theta) = \frac{1}{1 + \exp(-\mathbf{X} \cdot \theta)}
\end{aligned}
$$

So the same set of model architecture, loss function, and optimization algorithm can be used for both softmax regression and logistic regression. We will have one question covering the implementation of logistic regression algorithm in the homework assignment.
